Stamping files

In [1]:
from pathlib import Path
import os
from core import LocalFileFolderDataSource, VBaseProcessingPipeline, VBaseClient
from dotenv import load_dotenv

load_dotenv()
VBASE_API_URL = os.getenv("VBASE_API_URL")

In [2]:
VBASE_API_URL

'https://dev.app.vbase.com'

In [3]:
users_folder_path = "users"
users_folder = Path(users_folder_path).resolve()

assert VBASE_API_URL, "Missing VBASE_API_URL in .env"
assert users_folder.exists(), f"Users folder not found: {USERS_FOLDER}"

Load all users

In [5]:
data_source = LocalFileFolderDataSource(users_folder)
users_config = data_source.load_users()
users_config

{'users': {'user_id_1': {'api_key': 'wajlGePaeOaTri5hj1vfKpA--hJ_kXVjPK6FI84ovNs',
   'collections': {'collection_cid1': {'collection_cid': '0x64ad7103f0ef204d56f94d1923553d5dd7c1b9d5335dce70b21a7f44bd782712'},
    'collection_cid2': {'collection_cid': '0x83b8430d41237e1f120aa0b624214e880c742db4422d0f5f66f948c1b18a9602'}}}}}

Run stamping pipeline

In [6]:
def run_pipeline(api_url: str, users_folder: Path, run: bool, log_file: Path = None):
    if not users_folder.exists():
        raise ValueError(f"Folder does not exist: {users_folder}")

    data_source = LocalFileFolderDataSource(users_folder)
    users_config = data_source.load_users()

    for user_id, user_info in users_config["users"].items():
        print(f"\n=== Processing user: {user_id} ===")

        api_key = user_info.get("api_key")
        if not api_key:
            print(f"Skipping user {user_id}, missing API key.")
            continue

        stamping_client = VBaseClient(base_url=api_url, api_token=api_key)
        pipeline = VBaseProcessingPipeline(data_source, stamping_client)

        if run:
            pipeline.run(current_user=user_id)
            pipeline.print_summary()
            if log_file:
                pipeline.write_log(log_file)
                print(f"\nLog written to: {log_file}")
        else:
            preview = pipeline.preview(current_user=user_id)
            print("\n=== Preview Mode (use --run to execute stamping) ===")
            for entry in preview:
                print(f"{entry['file']} -> Collection: {entry['collection']} | User: {entry['user']}")

In [7]:
preview_run = True
api_url = os.getenv("VBASE_API_URL")

In [8]:
run_pipeline(api_url, users_folder, preview_run)


=== Processing user: user_id_1 ===

Processing collection: collection_cid1
collection_config.json -> Collection: collection_cid1 | Response: {'result': 'Idempotent request - returning previous stamp.', 'commitment_receipt': {'chainId': 421614, 'transactionHash': '0x11828718bb23f0a6b85a17bc3f300cb8aa5eb4548497db77791df9a7a8d226d2', 'user': 'cypress', 'objectCid': '0x429e7c9cf46fbfe4f4ed2fe68991de6b4bb01b02e40341588769d68b40b1f2f8', 'timestamp': '2025-06-23 19:13:54+00:00', 'setCid': '0x64ad7103f0ef204d56f94d1923553d5dd7c1b9d5335dce70b21a7f44bd782712', 'verified': True, 'verified_since': '2025-06-18T21:05:29.565Z', 'timedelta': '30 mins 52 secs ago', 'time': 'Jun-23-2025 07:13:54 PM +UTC', 'user_id': '0xb9CA15fF5da04B9dd30Da0C7124b84b623397cE0', 'collection_hash': '0x64ad7103f0ef204d56f94d1923553d5dd7c1b9d5335dce70b21a7f44bd782712', 'collection_name': 'collection_cid1', 'blockExplorerUrl': 'https://sepolia.arbiscan.io/tx/', 'blockchainName': 'Arbitrum Sepolia'}}
collection_cid1-test02.t